# [초급 프로젝트] 4팀_김명환

---
---

# 환경설정
    - 라이브러리 설치 및 로딩
    - 사용자 함수 사용

In [53]:
!pip install -q gdown
!pip install -q albumentations
!pip install -q ultralytics
!pip install -q -U ultralytics
!pip install -q nbformat
!pip install -q roboflow
!pip install -q opencv-python
!pip install -q opencv-python-headless
print("로딩완료")

로딩완료


In [54]:
# 기본 라이브러리 (중복 제거 및 정리)

# --- Scikit-learn: 데이터 전처리, 모델, 평가 ---
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.datasets import (
    fetch_california_housing, load_iris, make_moons, make_circles,
    load_breast_cancer, load_wine
)
from sklearn import datasets
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, mean_squared_error, average_precision_score

# --- 이미지 처리 ---
import cv2
from PIL import Image, ImageFilter, ImageDraw
import albumentations as A

# --- PyTorch: 딥러닝 관련 ---
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader, Subset
# 문제 있는 v2 import 제거하고 필요시에만 개별적으로 import
# from torchvision.transforms import v2, functional as TF
from torchvision.transforms import functional as TF
from torchvision.datasets import CocoDetection
from torch.nn import CrossEntropyLoss
from collections import OrderedDict

# --- COCO 데이터셋 관련 ---
from pycocotools.coco import COCO
from pycocotools import mask as coco_mask

# --- 딥러닝 모델 ---
import timm

# --- 기본 라이브러리 ---
import os
import sys
import re
import csv
import copy
import json
import math
import random
import yaml
import shutil
import requests
import xml.etree.ElementTree as ET
from pathlib import Path

# --- 데이터 분석 및 시각화 ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# --- 시간 관련 ---
from datetime import datetime, timezone, timedelta
import pytz

# --- 진행률 표시 ---
import IPython.display
from tqdm.notebook import tqdm

# --- 시간대 설정 ---
__kst = pytz.timezone('Asia/Seoul')

# --- GPU 설정 ---
__device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
__device_cpu = torch.device('cpu')

# --- 재현 가능한 결과를 위한 시드 설정 ---
np.random.seed(42)
torch.manual_seed(42)
if __device.type == 'cuda':
    torch.cuda.manual_seed_all(42)

print(f"라이브러리 로드 완료 사용장치: {__device}")

라이브러리 로드 완료 사용장치: cpu


In [55]:
from urllib.request import urlretrieve; urlretrieve("https://raw.githubusercontent.com/c0z0c/jupyter_hangul/refs/heads/beta/helper_c0z0c_dev.py", "helper_c0z0c_dev.py")
import importlib
import helper_c0z0c_dev as helper
importlib.reload(helper)

🌐 https://c0z0c.github.io/jupyter_hangul
ℹ️ NumPy 2.1.3 (v2.x+): 호환성 모드 적용됨
✅ 설정 완료: 한글 폰트, plt 전역 등록, pandas 확장, 캐시 기능
pd commit 저장 경로 = d:\GoogleDrive\codeit_ai_health_eat\scripts\김명환


<module 'helper_c0z0c_dev' from 'd:\\GoogleDrive\\codeit_ai_health_eat\\scripts\\김명환\\helper_c0z0c_dev.py'>

In [56]:
import os, sys
from pathlib import Path

utils_dir = None
if helper.is_colab:
    utils_dir = "/content/drive/MyDrive/codeit_ai_health_eat/src/python_modules/utils"
else:
    utils_dir = os.path.join(Path.cwd().drive + '\\', 'GoogleDrive', "codeit_ai_health_eat", "src", "python_modules", "utils")

print("utils_dir:", utils_dir)

sys.path.append(str(utils_dir))
print("sys.path:", sys.path)
import importlib
import health_ea_utils as heu
importlib.reload(heu)
from health_ea_utils import *

print("helper.__file__:", helper.__file__)
print("health_ea_utils.__file__:", heu.__file__)


utils_dir: d:\GoogleDrive\codeit_ai_health_eat\src\python_modules\utils
sys.path: ['D:\\GoogleDrive\\codeit_ai_health_eat\\script\\김명환\\ui', 'D:\\GoogleDrive\\codeit_ai_health_eat\\script\\김명환\\ui', 'D:\\GoogleDrive\\codeit_ai_health_eat\\script\\김명환\\ui', 'D:\\GoogleDrive\\codeit_ai_health_eat\\script\\김명환\\ui', 'D:\\GoogleDrive\\codeit_ai_health_eat\\script\\김명환', 'D:\\GoogleDrive\\codeit_ai_health_eat\\script\\김명환', 'D:\\GoogleDrive\\codeit_ai_health_eat\\script\\김명환', 'D:\\GoogleDrive\\codeit_ai_health_eat\\script\\김명환', 'D:\\GoogleDrive\\codeit_ai_health_eat\\script\\김명환', 'D:\\GoogleDrive\\codeit_ai_health_eat\\script\\김명환', 'D:\\GoogleDrive\\codeit_ai_health_eat\\script\\김명환', 'D:\\GoogleDrive\\codeit_ai_health_eat\\script\\김명환', 'D:\\GoogleDrive\\codeit_ai_health_eat\\script\\김명환', 'D:\\GoogleDrive\\codeit_ai_health_eat\\script\\김명환', 'D:\\GoogleDrive\\codeit_ai_health_eat\\script\\김명환', 'D:\\GoogleDrive\\codeit_ai_health_eat\\script\\김명환', 'D:\\GoogleDrive\\codeit_ai_health_ea

In [57]:
def get_tqdm_kwargs():
    """Widget 오류를 방지하는 안전한 tqdm 설정"""
    return {
        'disable': False,
        'leave': True,
        'file': sys.stdout,
        'ascii': True,  # ASCII 문자만 사용
        'dynamic_ncols': False,
#        'ncols': 80  # 고정 폭
    }

def drive_root():
    """
    Google Drive의 최상위 경로를 반환하는 함수입니다.
    - 로컬 환경(Windows): D:\GoogleDrive
    - Colab 환경: /content/drive/MyDrive
    프로젝트 내에서 데이터, 모델, 설정 파일 등 경로를 일관되게 관리할 때 사용합니다.
    """    
    root_path = os.path.join(Path.cwd().drive + '\\', "GoogleDrive")
    if helper.is_colab:
        root_path = os.path.join("/content/drive/MyDrive")
    return root_path

def get_path_modeling(add_path = None):
    """
    get_path_modeling() 함수는 모델링 관련 파일(예: 학습 결과, 체크포인트, 로그 등)을 저장할 경로를 반환합니다.
    기본적으로 Google Drive의 루트 경로(drive_root()) 아래 "modeling_yolo" 폴더를 기준으로 경로를 생성합니다.
    추가 하위 경로가 필요할 경우 add_path 인자를 통해 세부 폴더까지 지정할 수 있습니다.
    예시:
    get_path_modeling() → modeling_yolo (로컬)
    get_path_modeling("exp1") → D:\GoogleDrive\modeling_yolo\exp1
    Colab 환경에서는 /content/drive/MyDrive/modeling_yolo로 반환됩니다.
    """
    
    modeling_path = "modeling_yolo"
    path = os.path.join(drive_root(),modeling_path)
    if add_path is not None:
        path = os.path.join(path,add_path)
    return path

def get_path_modeling_release(add_path = None):
    """
    get_path_modeling_release() 함수는 모델링 결과물(예: 학습 결과, 체크포인트, 로그 등)을 저장할 경로를 반환합니다.
    기본적으로 Google Drive의 루트 경로(drive_root()) 아래 "modeling_yolo" 폴더를 기준으로 경로를 생성합니다.
    추가 하위 경로가 필요할 경우 add_path 인자를 통해 세부 폴더까지 지정할 수 있습니다.
    예시:
    get_path_modeling_release() → modeling_yolo (로컬)
    get_path_modeling_release("exp1") → D:\GoogleDrive\modeling_yolo\exp1
    Colab 환경에서는 /content/drive/MyDrive/modeling_yolo로 반환됩니다._
    """
    modeling_path = "modeling_yolo"
    path = os.path.join(drive_root(),modeling_path)
    if add_path is not None:
        path = os.path.join(path,add_path)
    return path

def save_model_dict(model, path, pth_name, kwargs=None):
    """모델 state_dict와 추가 정보를 저장"""
    def safe_makedirs(path):
        """안전한 디렉토리 생성"""
        if os.path.exists(path) and not os.path.isdir(path):
            os.remove(path)  # 파일이면 삭제
        os.makedirs(path, exist_ok=True)

    # 디렉토리 생성
    safe_makedirs(path)

    # 모델 구조 정보 추출
    model_info = {
        'class_name': model.__class__.__name__,
        'init_args': {},
        'str': str(model),
        'repr': repr(model),
        'modules': [m.__class__.__name__ for m in model.modules()],
    }

    # 생성자 인자 자동 추출(가능한 경우)
    if hasattr(model, '__dict__'):
        for key in ['in_ch', 'base_ch', 'num_classes', 'out_ch']:
            if hasattr(model, key):
                model_info['init_args'][key] = getattr(model, key)

    # kwargs 처리
    extra_info = {}
    if kwargs is not None:
        if isinstance(kwargs, str):
            extra_info = json.loads(kwargs)
        elif isinstance(kwargs, dict):
            extra_info = kwargs

    model_info.update(extra_info)

    # 저장할 dict 구성
    save_dict = {
        'model_state': model.state_dict(),
        'class_name': model.__class__.__name__,
        'model_info': model_info,
    }

    save_path = os.path.join(path, f"{pth_name}.pth")
    torch.save(save_dict, save_path)
    return save_path

def load_model_dict(path, pth_name=None):
    """
    save_model_dict로 저장한 모델을 불러오는 함수
    반환값: (model_state, model_info)
    """
    import torch
    load_path = path
    if pth_name is not None:
        load_path = os.path.join(path, f"{pth_name}.pth")
    checkpoint = torch.load(load_path, map_location='cpu', weights_only=False)  # <-- 여기 추가
    model_state = checkpoint.get('model_state')
    model_info = checkpoint.get('model_info')
    model_info['file_name'] = os.path.basename(load_path)
    return model_state, model_info


def search_pth_files(base_path):
    """
    입력된 경로의 하위 폴더들에서 pth 파일들을 검색
    """
    pth_files = []

    if not os.path.exists(base_path):
        print(f"경로가 존재하지 않습니다: {base_path}")
        return pth_files

    print(f"pth 파일 검색 시작: {base_path}")

    # 하위 폴더들을 순회하며 pth 파일 검색
    for root, dirs, files in os.walk(base_path):
        for file in files:
            if file.endswith('.pth'):
                pth_path = os.path.join(root, file)
                pth_files.append(pth_path)

    # 결과 정리 및 출력
    if pth_files:
        print(f"\n발견된 pth 파일들 ({len(pth_files)}개):")
        for i, pth_file in enumerate(pth_files, 1):
            # 상대 경로로 표시 (base_path 기준)
            rel_path = os.path.relpath(pth_file, base_path)
            print(f" {i:2d}. {rel_path}")
    else:
        print("pth 파일을 찾을 수 없습니다.")

    return pth_files

def print_dir_tree(root, max_depth=2, list_count=3, indent=""):
    """
    지정한 폴더(root) 하위의 디렉토리 구조를 트리 형태로 출력하는 함수입니다.

    Args:
        root: 시작 경로(폴더)
        max_depth: 출력할 최대 깊이(디폴트 2)
        list_count: 파일 개수가 많을 때 몇 개만 출력할지(디폴트 3)
        indent: 들여쓰기(재귀적으로 사용)
    """
    import os
    if max_depth < 0:
        return
    try:
        items = os.listdir(root)
    except Exception as e:
        print(indent + f"[Error] {e}")
        return

    img_count = len([f for f in os.listdir(root) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.xml', '.inf', '.txt'))])
    for item in items:
        path = os.path.join(root, item)
        if os.path.isdir(path):
            print(indent + "|-- "+ item)
            # 이미지 파일 개수만 출력
            img_count = len([f for f in os.listdir(path) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.xml', '.inf', '.txt'))])
            if img_count > list_count:
                print(indent + "   "+ f"[데이터파일: {img_count}개]")
            print_dir_tree(root=path, max_depth=max_depth-1, list_count=list_count, indent=indent + "   ")
        else:
            if list_count < img_count and item.lower().endswith(('.jpg', '.jpeg', '.png', '.xml', '.inf', '.txt')):
                continue
            print(indent + "|-- "+ item)

def print_json_tree(data, indent="", max_depth=4, _depth=0, list_count=2, print_value=True, max_length=30):
    """
    JSON 객체를 지정한 단계(max_depth)까지 트리 형태로 출력
    - list 타입은 3개 이상일 때 개수만 출력
    - 하위 노드가 값일 경우 key(type) 형태로 출력
    - print_value=True일 때 key(type): 값 형태로 출력
    """
    if _depth > max_depth:
        return
    if isinstance(data, dict):
        for key, value in data.items():
            if isinstance(value, (dict, list)):
                print(f"{indent}|-- {key}")
                print_json_tree(value, indent + "    ", max_depth, _depth + 1, list_count, print_value)
            else:
                if print_value:
                    print(f"{indent}|-- {key}({type(value).__name__}): {value if len(str(value)) < 100 else f'{str(value)[:max_length]}...'}")
                else:
                    print(f"{indent}|-- {key}({type(value).__name__})")
    elif isinstance(data, list):
        if len(data) > list_count:
            print(f"{indent}|-- [list] ({len(data)} items)")
        else:
            for i, item in enumerate(data):
                if isinstance(item, (dict, list)):
                    print(f"{indent}|-- [{i}]")
                    print_json_tree(item, indent + "    ", max_depth, _depth + 1, list_count, print_value)
                else:
                    if print_value:
                        print(f"{indent}|-- [{i}]({type(item).__name__}): {item if len(str(item)) < max_length else f'{str(item)[:max_length]}...'}")
                    else:
                        print(f"{indent}|-- [{i}]({type(item).__name__})")
    else:
        if print_value:
            print(f"{indent}{type(data).__name__}: {data if len(str(data)) < max_length else f'{str(data)[:max_length]}...'}")
        else:
            print(f"{indent}{type(data).__name__}")

def print_git_tree(data, indent="", max_depth=3, _depth=0):
    """
    PyTorch tensor/딕셔너리/리스트를 git tree 스타일로 출력
    """
    import torch
    import numpy as np

    if _depth > max_depth:
        return
    if isinstance(data, dict):
        for key, value in data.items():
            print(f"{indent}├─ {key} [{type(value).__name__}]")
            print_git_tree(value, indent + "│  ", max_depth, _depth + 1)
    elif isinstance(data, (list, tuple)):
        for i, item in enumerate(data):
            print(f"{indent}├─ [{i}] [{type(item).__name__}]")
            print_git_tree(item, indent + "│  ", max_depth, _depth + 1)
    elif torch.is_tensor(data):
        shape = tuple(data.shape)
        dtype = str(data.dtype)
        preview = str(data)
        preview_str = preview[:80] + ("..." if len(preview) > 80 else "")
        print(f"{indent}└─ Tensor shape={shape} dtype={dtype} preview={preview_str}")
    elif isinstance(data, np.ndarray):
        shape = data.shape
        dtype = data.dtype
        preview = str(data)
        preview_str = preview[:80] + ("..." if len(preview) > 80 else "")
        print(f"{indent}└─ ndarray shape={shape} dtype={dtype} preview={preview_str}")
    else:
        val_str = str(data)
        print(f"{indent}└─ {type(data).__name__}: {val_str[:80]}{'...' if len(val_str)>80 else ''}")


print("유틸리티 함수 로드 완료")

유틸리티 함수 로드 완료


In [58]:
from pathlib import Path
from ultralytics import YOLO

# GPU 확인
device = __device
print(f"사용 디바이스: {device}")
print(f"CUDA 버전: {torch.version.cuda}")

사용 디바이스: cpu
CUDA 버전: None


# 프로그래밍

## PillAnalysisEngine 모듈 로드 및 테스트

`PillAnalysisEngine`은 이미지 내 알약(약물)을 탐지하고, 각 알약의 종류를 분류한 뒤, 병용 금기 약물(함께 복용하면 안 되는 약물 조합) 여부를 체크하는 엔진입니다.

주요 기능 요약:

- **알약 객체 탐지:** 입력된 이미지에서 알약을 찾아냅니다.
- **알약 종류 분류:** 탐지된 각 알약의 종류(약품명 등)를 분류합니다.
- **병용 금기 약물 체크:** 분류된 알약들 간에 병용 금기 조합이 있는지 검사하여 결과를 제공합니다.

즉, 이미지 한 장을 입력하면, 그 안의 알약들을 자동으로 인식하고, 각각의 정보를 추출하며, 복용 시 주의해야 할 조합까지 알려주는 약물 분석 자동화 도구입니다.

In [59]:
import os, sys
from pathlib import Path
user_dir = Path(drive_root()) / "codeit_ai_health_eat" / "script" / "김명환"
sys.path.insert(0, str(user_dir.resolve()))
import PillAnalysisEngine as PillAnalysisEngine
import importlib; importlib.reload(PillAnalysisEngine)
from PillAnalysisEngine import PillAnalysisEngine


In [60]:
pillEngine = PillAnalysisEngine()

d:\GoogleDrive\codeit_ai_health_eat
d:\GoogleDrive\codeit_ai_health_eat\src\python_modules\modeling
d:\GoogleDrive\codeit_ai_health_eat\src\python_modules\data


True d:\GoogleDrive\codeit_ai_health_eat\src\python_modules\modeling\yolo\yolov8m_yolo_noresize_one_class_20250915_0858\weights\best.pt
True d:\GoogleDrive\codeit_ai_health_eat\src\python_modules\modeling\efficientnet_b3\efficientnet_b3_experiment_20250915_224641\best.pth


- 1.png
    - 알약 객체를 탐지
    - 알약 종류 분류
    - 알약 병용 금기 약물 체크

In [61]:
impage_path = r'D:\dataset\kaggle_code_it_data\ai04-level1-project.zip.unzip\test_images\1.png'
results_json = pillEngine.analyze_image(impage_path)
#print(results_json )
print_json_tree(results_json, max_depth=5, list_count=3, print_value=True, max_length=1000)
# results_dict = json.loads(results_json)
# print(results_dict)
# print(results_dict['classifications'])
# for cls in results_dict['classifications']:
#     print(cls['drug_info'])
    #print(json.dumps(cls, ensure_ascii=False, indent=2))
    #print(f"{cls['class_id']:2d} {str(cls['class_name']):20} {cls['box']} {cls['probabilitie']:.4f}")

str: {
  "ddi": [],
  "ddi_drug": [
    {
      "성분명A": "atazanavir sulfate (as atazanavir)",
      "성분코드A": "458503ACH",
      "제품코드A": 658700040,
      "제품명A": "레야타즈캡슐200밀리그램(아타자나비어황산염)_(0.22779g/1캡슐)",
      "업체명A": "(유)한국비엠에스제약",
      "급여구분A": "급여",
      "성분명B": "ilaprazole",
      "성분코드B": "505501ATE",
      "제품코드B": 641702920,
      "제품명B": "놀텍정10밀리그램(일라프라졸)_(10mg/1정)",
      "업체명B": "일양약품(주)",
      "급여구분B": "급여",
      "공고번호": 20100246,
      "공고일자": "2010-11-02",
      "금기사유": "아타자나비르의 혈중농도 감소",
      "category_id": 24849
    },
    {
      "성분명A": "atazanavir sulfate (as atazanavir)+cobicistat silicon dioxide (as cobicistat)",
      "성분코드A": "646700ATB",
      "제품코드A": 658700670,
      "제품명A": "에보타즈정_(1정)",
      "업체명A": "(유)한국비엠에스제약",
      "급여구분A": "급여",
      "성분명B": "ilaprazole",
      "성분코드B": "505501ATE",
      "제품코드B": 641702920,
      "제품명B": "놀텍정10밀리그램(일라프라졸)_(10mg/1정)",
      "업체명B": "일양약품(주)",
      "급여구분B": "급여",
      "공고번호": 20230045,
      "공고일자": "2023-07-05